<a href="https://colab.research.google.com/github/basugautam/Reproducibility-Challenge-Project/blob/Architecture-Files/5_primal_dual_algorithm_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 🔹 Step 1: Load Dataset
dataset_path = r"C:\LCTSF\Dataset\timeseries_dataset.csv"  # Dataset path
df = pd.read_csv(dataset_path)

# 🔹 Step 2: Preprocessing
data = df['value'].values.reshape(-1, 1)  # Reshape for scaling
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# 🔹 Step 3: Create Sequences for Time Series Forecasting
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, 0])  # Input sequence
        y.append(data[i+seq_length, 0])    # Target output
    return np.array(X), np.array(y)

sequence_length = 50  # Define sequence window size
X, y = create_sequences(data_scaled, sequence_length)

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Reshape data for LSTM input format
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# 🔹 Step 4: Define LSTM Model (Primal Problem)
model = Sequential([
    LSTM(64, activation='relu', return_sequences=True, input_shape=(sequence_length, 1)),
    Dropout(0.2),
    LSTM(32, activation='relu', return_sequences=False),
    Dense(1)
])

# 🔹 Step 5: Define the Primal-Dual Loss Function
class PrimalDualLoss(tf.keras.losses.Loss):
    """
    Custom loss function implementing the Primal-Dual Algorithm.

    - Balances the forecasting accuracy with constraint enforcement.
    - Uses Lagrange multipliers (lambda) to dynamically adjust constraints.
    """
    def __init__(self, lambda_dual=0.1, max_error=0.05):
        super(PrimalDualLoss, self).__init__()
        self.lambda_dual = tf.Variable(lambda_dual, trainable=False, dtype=tf.float32)  # Dual variable
        self.max_error = max_error  # Upper bound for error

    def call(self, y_true, y_pred):
        error = tf.abs(y_true - y_pred)  # Absolute forecasting error
        primal_loss = tf.reduce_mean(tf.square(error))  # MSE loss (primal)
        dual_penalty = tf.reduce_mean(tf.maximum(error - self.max_error, 0) ** 2)  # Dual penalty
        total_loss = primal_loss + self.lambda_dual * dual_penalty  # Combined Primal-Dual Loss
        return total_loss

# Initialize loss function
primal_dual_loss = PrimalDualLoss(lambda_dual=0.1, max_error=0.05)

# 🔹 Step 6: Compile and Train the Model
model.compile(optimizer=Adam(learning_rate=0.001), loss=primal_dual_loss)
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# 🔹 Step 7: Implement Primal-Dual Algorithm Update
def primal_dual_update(model, X_train, y_train, primal_dual_loss, learning_rate=0.001, lambda_step=0.01):
    """
    Implements the Primal-Dual Algorithm:
    - Updates primal variables (LSTM weights)
    - Adjusts dual variables (Lagrange multipliers)
    """
    optimizer = Adam(learning_rate=learning_rate)

    for epoch in range(10):  # Iterate over multiple updates
        with tf.GradientTape() as tape:
            y_pred = model(X_train, training=True)
            loss_value = primal_dual_loss(y_train, y_pred)

        gradients = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # 🔹 Step 8: Update Lagrange Multipliers (Dual Variables)
        primal_dual_loss.lambda_dual.assign(primal_dual_loss.lambda_dual + lambda_step * tf.reduce_mean(tf.maximum(loss_value - primal_dual_loss.max_error, 0)))

        print(f"Epoch {epoch+1}: Loss = {loss_value.numpy()}, Lambda = {primal_dual_loss.lambda_dual.numpy()}")

# 🔹 Step 9: Run the Primal-Dual Optimization
primal_dual_update(model, X_train, y_train, primal_dual_loss)

# 🔹 Step 10: Evaluate the Model
y_pred = model.predict(X_test)

# Rescale predictions to original scale
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_rescaled = scaler.inverse_transform(y_pred)

# 🔹 Step 11: Plot Predictions vs Actual Values
plt.figure(figsize=(10, 5))
plt.plot(y_test_rescaled, label="Actual", color="blue")
plt.plot(y_pred_rescaled, label="Predicted", color="red", linestyle="dashed")
plt.title("Primal-Dual Forecasting Optimization")
plt.xlabel("Time Steps")
plt.ylabel("Value")
plt.legend()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\LCTSF\\Dataset\\timeseries_dataset.csv'